In [8]:
import os

import pandas as pd
import json
import random

from subword_prompt_templates.MultiChoicePrompts import MultiChoicePrompts
from subword_prompt_templates.ClassificationPrompts import ClassificationPrompts

from utils import helpers

from openai import OpenAI

In [2]:
models = ['MISTRALAI/MIXTRAL-8X7B-INSTRUCT-V0.1', 'allenai/OLMo-7B-Instruct', 'meta-llama/Llama-3-8b-chat-hf']
prompts = ['zero_shot', 'one_shot', 'few_shot', 'cot\one', 'cot\few', 'decomposite']
dataset = pd.read_csv(r"datasets\dataset_for_evaluation.csv", index_col=0)

TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")
client = OpenAI(api_key=TOGETHER_API_KEY, base_url='https://api.together.xyz/v1')

random.seed(42)

# multi-chioce

In [7]:
results = []
mcp = MultiChoicePrompts()
ynp = ClassificationPrompts()
for model in models:
    for prompt_type in prompts:
        for row in dataset.itertuples():
            wrong_choices = json.loads(row.Mulitple_Options.replace(" ", ", ").replace("'", '"'))
            correct_choice = row.Word
            choices = wrong_choices + [correct_choice]
            random.shuffle(choices)
            correct_choice_letter = chr(choices.index(correct_choice) + 65)
            multi_question = mcp.generate(prompt_type, row.Category, *choices)
            yes_question = ynp.generate(prompt_type, row.Category, row.Word)
            no_question = ynp.generate(prompt_type, row.Category, random.choice(wrong_choices))
            multi_answer = helpers.generate_llm_response(client, multi_question, model)
            yes_answer = helpers.generate_llm_response(client, yes_question, model)
            no_answer = helpers.generate_llm_response(client, no_question, model)
            multi_eval = helpers.evaluate_response('multi', multi_answer, correct_choice_letter)
            yes_eval = helpers.evaluate_response('classification', yes_answer, "Yes")
            no_eval = helpers.evaluate_response('classification', no_answer, "No")
            results.append((model, prompt_type, 'multi', multi_question, multi_answer, correct_choice_letter, multi_eval))
            results.append((model, prompt_type, 'classification', yes_question, yes_answer, "yes", yes_eval))
            results.append((model, prompt_type, 'classification', no_question, no_answer, "No", no_eval))
            break
        break
    break
pd.DataFrame(results, columns=["Model", "Prompt Type", "Question type", "Question", "LLM Response", "Correct answer", "Eval"])#.to_csv("response_list.csv")

,Model,Prompt Type,Question type,Question,LLM Response,Correct answer,Eval
0,MISTRALAI/MIXTRAL-8X7B-INSTRUCT-V0.1,zero_shot,multi,Subword is a standalone word that exists withi...,"A. Incredible - The subword ""red"" can be foun...",A,True
1,MISTRALAI/MIXTRAL-8X7B-INSTRUCT-V0.1,zero_shot,classification,Subword is a standalone word that exists withi...,"Yes, the word ""incredible"" contains the subwo...",yes,True
2,MISTRALAI/MIXTRAL-8X7B-INSTRUCT-V0.1,zero_shot,classification,Subword is a standalone word that exists withi...,"Yes, the word ""lower"" contains the subword ""r...",No,False


# Yes\No

In [7]:
import re

gold_response = "A"

pattern = rf'\W*{re.escape(gold_response)}\W*\b'

# Example strings to test
test_strings = ["A.", "A)", "[A]", "A", "An", "A+", "A*", "A ", " A ", "The A", "A is the correct answer"]

# Check each string
for string in test_strings:
    if re.match(pattern, string):
        print(f"'{string}' starts with '{gold_response}'")
    else:
        print(f"'{string}' does not match the pattern.")

'A.' starts with 'A'
'A)' starts with 'A'
'[A]' starts with 'A'
'A' starts with 'A'
'An' does not match the pattern.
'A+' starts with 'A'
'A*' starts with 'A'
'A ' starts with 'A'
' A ' starts with 'A'
'The A' does not match the pattern.
'A is the correct answer' starts with 'A'


In [9]:
import re

gold_response = "Yes"

pattern = rf'\W*{gold_response}\b'

# Example strings to test
test_strings = ["Yes.", "Yes)", "[Yes]", "Yes", "Yess", "Yes ", " Yes ", "The Yes", "Yes is the correct answer"]

# Check each string
for string in test_strings:
    if re.match(pattern, string):
        print(f"'{string}' starts with '{gold_response}'")
    else:
        print(f"'{string}' does not match the pattern.")
        

'Yes.' starts with 'Yes'
'Yes)' starts with 'Yes'
'[Yes]' starts with 'Yes'
'Yes' starts with 'Yes'
'Yess' does not match the pattern.
'Yes ' starts with 'Yes'
' Yes ' starts with 'Yes'
'The Yes' does not match the pattern.
'Yes is the correct answer' starts with 'Yes'
